In [1]:
import sys

import matplotlib.pyplot as plt

import torch

from iopath.common.file_io import PathManager
from iopath.fb.manifold import ManifoldPathHandler

sys.path.insert(0, "fbcode/ar_sharing/body_tracking_3d/models/hmr/")

pathmgr = PathManager()
pathmgr.register_handler(ManifoldPathHandler(), allow_override=True)

%load_ext autoreload
%autoreload 2

I1006 062410.137 _utils_internal.py:179] NCCL_DEBUG env var is set to None


I1006 062410.139 _utils_internal.py:188] NCCL_DEBUG is INFO from /etc/nccl.conf


I1006 062411.183 parsh_logger.py:29] %autoreload 2 was called with 


In [2]:
import src

In [3]:
from src.functional import smpl
from src.functional.optical_flow import (
    get_corners_for_points,
    unproject_corners_on_mesh,
    unproject_data_to_vertices,
)
from src.functional.renderer import (
    ColoredRenderer,
    convert_vertices_to_mesh,
    fit_vertices_to_orthographic,
    get_default_cameras,
)
from src.models.seq_optim import SeqModel

In [8]:
torch.set_grad_enabled(False)

### load sample
seqOpt = SeqModel(
    **{
        "seq_path": "manifold://xr_body/tree/personal/andreydavydov/3dpw_seq_for_tests/sample_3dpw_test_seq_downtown_runForBus_01_0__hmr_coco_all.pth",
        "seq_num_frames": 339,
        "optical_flow_presaved": "manifold://xr_body/tree/personal/andreydavydov/3dpw_seq_for_tests/optical_flow_presaved/optical_flows_for_seq.pth",
    }
)

smpl_model = smpl.get_smpl_model()
smpl_model_faces = smpl_model.faces.astype(int)
smpl_model_faces = torch.tensor(smpl_model_faces.copy()).unsqueeze(0)

opt_flow_forward = seqOpt.optical_flow["forward_time"]
opt_flow_backward = seqOpt.optical_flow["backward_time"]

### keep only 10 frames
B = 10
seqOpt.pose.data = seqOpt.pose.data[:B]
seqOpt.shape.data = seqOpt.shape.data[:B]
seqOpt.camera.data = seqOpt.camera.data[:B]
opt_flow_forward = opt_flow_forward[: B - 1]
opt_flow_backward = opt_flow_backward[: B - 1]
img_size = opt_flow_forward.size(-1)

device = "cuda:0"
seqOpt = seqOpt.to(device)
opt_flow_forward = opt_flow_forward.to(device)
opt_flow_backward = opt_flow_backward.to(device)
smpl_model = smpl_model.to(device)
cameras = get_default_cameras(device, mode="orthographic")

### align vertices with pixels
_, _, verts3d = seqOpt(smpl_model)
batch_size = verts3d.size(0)
camera = seqOpt.camera
scale, trans = camera[:, 0], camera[:, 1:]
verts3d = verts3d * scale.view(batch_size, 1, 1)
verts3d[:, :, 0:2] = verts3d[:, :, 0:2] + trans.view(batch_size, 1, 2)
verts3d = (verts3d + 1) / 2 * img_size

I1006 062504.718 manifold.py:1442] URL manifold://xr_body/tree/personal/andreydavydov/3dpw_seq_for_tests/sample_3dpw_test_seq_downtown_runForBus_01_0__hmr_coco_all.pth was already cached in /var/svcscm/.torch/iopath_cache/manifold_cache/tree/personal/andreydavydov/3dpw_seq_for_tests/sample_3dpw_test_seq_downtown_runForBus_01_0__hmr_coco_all.pth


I1006 062505.073 manifold.py:1442] URL manifold://xr_body/tree/personal/andreydavydov/3dpw_seq_for_tests/optical_flow_presaved/optical_flows_for_seq.pth was already cached in /var/svcscm/.torch/iopath_cache/manifold_cache/tree/personal/andreydavydov/3dpw_seq_for_tests/optical_flow_presaved/optical_flows_for_seq.pth


I1006 062506.632 manifold.py:1442] URL manifold://xr_body/tree/personal/andreydavydov/eft/extradata/smpl/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl was already cached in /var/svcscm/.torch/iopath_cache/manifold_cache/tree/personal/andreydavydov/eft/extradata/smpl/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl


I1006 062506.737 manifold.py:1442] URL manifold://xr_body/tree/personal/andreydavydov/eft/extradata/data_from_spin/J_regressor_extra.npy was already cached in /var/svcscm/.torch/iopath_cache/manifold_cache/tree/personal/andreydavydov/eft/extradata/data_from_spin/J_regressor_extra.npy


In [9]:
### prepare verts3d for start/end mesh vertices
verts3d_start, verts3d_end = verts3d[:-1], verts3d[1:]

### unproject OF onto M1
unproj_flow2d_forward, vis_mask_start = unproject_data_to_vertices(
    verts3d[:-1], opt_flow_forward, smpl_model_faces, cameras
)
vis_mask_start = vis_mask_start.view(-1)

print(verts3d_start.shape, verts3d_end.shape)
print(vis_mask_start.shape)
print(unproj_flow2d_forward.shape)

torch.Size([9, 6890, 3]) torch.Size([9, 6890, 3])
torch.Size([62010])
torch.Size([9, 6890, 2])


In [12]:
### compute displacements for Mesh1 vertices on Image2
m1_on_i2 = verts3d_start[..., :2] + unproj_flow2d_forward  # (B-1) x N x 2
print(m1_on_i2.shape)

px_quads, dx, dy = get_corners_for_points(m1_on_i2)
bary_coords_corners, verts_triplet_indices, vis_mask_end = unproject_corners_on_mesh(px_quads, verts3d[1:], smpl_model_faces, cameras, img_size=(img_size,img_size))

torch.Size([9, 6890, 2])


In [13]:
vis_mask = (vis_mask_start * vis_mask_end).type(torch.bool)  # mask of verts of M1 that go to visible verts of M2
print(bary_coords_corners.shape, verts_triplet_indices.shape, vis_mask.shape)

m1_vis_indices = torch.arange(len(vis_mask)).to(device)[vis_mask].view(-1, 1)
m2_vis_indices_all = verts_triplet_indices.reshape(-1, 12)[vis_mask]
m_map_to_same_indices = (m2_vis_indices_all == m1_vis_indices).type(torch.bool)

bary_coords_corners = bary_coords_corners[vis_mask].reshape(-1, 12)

print(m1_vis_indices.shape, m2_vis_indices_all.shape, m_map_to_same_indices.shape, bary_coords_corners.shape)
print(m1_vis_indices[509])
print(m2_vis_indices_all[509])
print(m_map_to_same_indices[509])

torch.Size([62010, 4, 3]) torch.Size([62010, 4, 3]) torch.Size([62010])
torch.Size([6663, 1]) torch.Size([6663, 12]) torch.Size([6663, 12]) torch.Size([6663, 12])
tensor([5264], device='cuda:0')
tensor([6573, 5261, 5264, 5264, 4354, 5262, 5263, 6566, 5264, 5262, 5263, 5264],
       device='cuda:0')
tensor([False, False,  True,  True, False, False, False, False,  True, False,
        False,  True], device='cuda:0')


In [18]:
dx = dx.view(-1)[vis_mask]
dy = dy.view(-1)[vis_mask]
w_tl = dx * dy
w_tr = (1 - dx) * dy
w_bl = dx * (1 - dy)
w_br = (1 - dx) * (1 - dy)
w_faces = torch.stack((w_tl, w_tr, w_bl, w_br), dim=-1)
w_faces = w_faces[..., None].repeat(1, 1, 3).view(-1, 12)

w = bary_coords_corners * w_faces
print(w.shape)

torch.Size([6663, 12])


In [19]:
w[~m_map_to_same_indices]

tensor([0.0487, 0.1440, 0.0283,  ..., 0.1310, 0.0188, 0.0942], device='cuda:0')

In [21]:
m_map_to_same_indices

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]], device='cuda:0')

In [ ]:
sample = torch.load(pathmgr.get_local_path("manifold://xr_body/tree/personal/andreydavydov/renderer/sample_for_tests.pth"))

In [ ]:
device = "cuda:0"

img = sample["img"].to(device)
vertices = sample["bbox_verts"].to(device)
faces = sample["faces"]

batch_size = img.size(0)
img_size = img.shape[-2:]

faces_batch = torch.tensor(faces.copy()).unsqueeze(0).repeat(batch_size, 1, 1).to(device)

vertices = fit_vertices_to_orthographic(vertices, img_size)
meshes = convert_vertices_to_mesh(vertices, faces_batch)

renderer = ColoredRenderer(img_size=img_size, device=device, diffuse_color=True, specular_color=True)

In [275]:
fragments = renderer.rasterizer(meshes)
pix_to_face = fragments.pix_to_face
bary_coords = fragments.bary_coords
pix_to_face_mask = pix_to_face >= 0

NameError: name 'renderer' is not defined

In [276]:
b = 8

In [277]:
print(pix_to_face.shape, bary_coords.shape)
print(pix_to_face[b, 98:101, 174:177, 0])
print(bary_coords[b, 98:101, 174:177, 0]) # barycentric coords for each of vertices (I guess)

h, w = 99, 175

torch.Size([5, 224, 224, 1]) torch.Size([5, 224, 224, 1, 3])


IndexError: index 8 is out of bounds for dimension 0 with size 5

In [ ]:
tri = pix_to_face[b, h, w, 0]
print(tri)
verts_triplet_indices = faces_batch.view(-1, 3)[tri]
print(verts_triplet_indices)

In [ ]:
vertices[b][verts_triplet_indices]  # N=3 x xyz=3

In [ ]:
p = (bary_coords[b, h, w, 0].view(3,1) * vertices[b][verts_triplet_indices]).sum(dim=0) # A,B,C with bary coords - must be done for every img pixel!
print(p)

In [ ]:
(-p[:2] + 1) / 2 * 224

In [278]:
plt.imshow(renderer(meshes)[0].cpu())

NameError: name 'renderer' is not defined